In [4]:
import os

# Define root directory where the simulation files are located
root_directory = "/n17data/tersenov/SLICS/Cosmo_DES"

# Directories to exclude
exclude_dirs = ["SLICS_HR", "fid_a", "fid_f"]

# Open the master text file in write mode
master_file_path = "master_file.txt"
with open(master_file_path, "w") as master_file:
    # Iterate over the files and subdirectories in the root directory
    for root, dirs, files in os.walk(root_directory):
        # Exclude the specified directories
        dirs[:] = [d for d in dirs if d not in exclude_dirs]
        
        for file_name in files:
            # Write the file path to the master file
            file_path = os.path.join(root, file_name)
            master_file.write(file_path + "\n")

In [5]:
import numpy as np

def process_files(file_paths):
    """Reads the information in the filename.

    Parameters
    ----------
    file_paths : list of str (.txt file)
        List containing the paths to the files to be processed

    Returns
    -------
    data : recarray
        Numpy recarray containing the information extracted from the file names

    """

    # Make empty recarray to store the data
    data = np.recarray(len(file_paths), dtype=[('id', int), ('seed', 'U1'), ('bin', int), ('LOS', int), ('tile', int)])

    # Iterate over the file paths and process each file
    for i, file_path in enumerate(file_paths):
        # Extract the file name from the file path
        file_name = file_path.split("/")[-1]
        
        # Split file name into parts
        file_parts = file_name.split("_")
        
        id = int(file_parts[2])
        seed = file_parts[3]
        unknown_number = int(file_parts[4])
        bin = int(file_parts[5][3:])  # Extract the number after "Bin"
        LOS = int(file_parts[6][3:])  # Extract the number after "LOS"
        tile = int(file_parts[7][1:-4])  # Extract the number after "R"

        # Assign the extracted data to the corresponding fields in the recarray
        data[i]['id'] = id
        data[i]['seed'] = seed
        data[i]['bin'] = bin
        data[i]['LOS'] = LOS
        data[i]['tile'] = tile
        # print(data[i])

    return data 


In [6]:
# Read the file paths from master_file.txt
filename = "master_file.txt"
with open(filename, 'r') as file:
    file_paths = file.readlines()
    file_paths = [path.strip() for path in file_paths]

data = process_files(file_paths)

In [7]:
def read_cosmo_params(file_path):
    """Reads the cosmological parameters from the .dat file.

    Parameters
    ----------
    file_path : str
        Path to the .dat file containing the cosmological parameters

    Returns
    -------
    cosmo_params : dict
        Dictionary mapping each ID to its corresponding cosmological parameters

    """

    cosmo_params = {}

    with open(file_path, 'r') as file:
        lines = file.readlines()

        # Iterate over the lines starting from the second line
        for line in lines[1:]:
            line = line.strip()
            if line:
                parts = line.split()
                id = int(parts[0])
                params = {
                    'Om': float(parts[1]),
                    'h': float(parts[2]),
                    'w_0': float(parts[3]),
                    'sigma_8': float(parts[4]),
                    'Oc': float(parts[5])
                }
                cosmo_params[id] = params

    return cosmo_params

In [8]:
# Path to the .dat file
dat_file_path = "/home/tersenov/shear-pipe-peaks/example/CosmoTable.dat"

# Read the cosmological parameters from the .dat file
cosmo_params = read_cosmo_params(dat_file_path)

# Map the IDs in the recarray to the corresponding cosmological parameters
mapped_params = []
for row in data:
    id = row['id']
    params = cosmo_params.get(id)
    if params:
        mapped_params.append(params)
    else:
        print(f"No parameters found for ID {id}")

# Now, 'mapped_params' will contain the corresponding cosmological parameters for each ID in the recarray

# Generate the mass map from the catalog

In [9]:
import numpy as np

# Define the path to the catalog file
catalog_file = "/n17data/tersenov/SLICS/Cosmo_DES/00_a/LOS1/DES_MocksCat_00_a_4_Bin1_LOS1_R1.dat"

# Load the catalog data as a numpy array
catalog_data = np.loadtxt(catalog_file)

# Print the loaded data
print(catalog_data)


[[ 8.9438686e+00  7.3266211e+00 -2.6491606e-01 ...  1.3698399e-03
  -2.0684400e-03  8.9141685e-01]
 [ 9.2625837e+00  7.2931900e+00 -5.3840622e-02 ...  1.1668235e-05
  -7.9777034e-04  7.6248813e-01]
 [ 9.1085062e+00  7.3634996e+00 -2.3904347e-01 ...  1.4143117e-03
  -2.6327133e-04  8.1961751e-01]
 ...
 [ 5.3041000e+00  6.0832686e+00  2.6433794e-02 ...  5.1106932e-04
  -2.6381079e-03  7.4370402e-01]
 [ 5.0223250e+00  6.0880365e+00  5.4065637e-02 ...  1.3714051e-02
   2.6118999e-02  7.0694596e-01]
 [ 5.3476853e+00  6.0835223e+00  3.8928293e-02 ...  3.2676180e-04
  -3.0681316e-03 -1.3368914e-01]]


In [10]:
catalog_data.shape

(272143, 10)